# Capstone

Michael Schillawski, 10 April 2018

Data Science Immersive, General Assembly

## Imports

In [1]:
import os
import json
import re
import string
import multiprocessing
import pickle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets

from pandas.io.json import json_normalize
from joblib import Parallel, delayed
from tqdm import tqdm
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.cluster.hierarchy import dendrogram, linkage, cophenet, fcluster
from scipy.spatial.distance import pdist
from __future__ import print_function
from ipywidgets import interact, interactive, fixed
from ipywidgets import *

%matplotlib inline

os.getcwd()

'/Users/mjschillawski/Google Drive/Data/generalassembly/projects/GitHub Portfolio/capstone_project'

## Load Data

### Load Data From JSON

In [ ]:
path = '/Users/mjschillawski/Desktop/Miscellaneous Data/Yummly28K/'
file = 'data_records_27638.txt'

data = pd.read_table(path+file,header=None,names=['recipe'],index_col=1)

In [ ]:
path = '/Users/mjschillawski/Desktop/Miscellaneous Data/Yummly28K/metadata27638/'

recipes = []

for i in data.index:
    num = str(i)
    while len(num) < 5:
        num = '0' + num
        
    # https://stackoverflow.com/questions/28373282/how-to-read-a-json-dictionary-type-file-with-pandas
    with open(path+'meta'+num+'.json') as json_data:
        recipe = json.load(json_data)
        recipes.append(recipe)

recipes = json_normalize(recipes)

recipes.to_csv('assets/recipes_dataset.csv')

### Load Data From CSV

In [2]:
recipes = pd.read_csv('assets/recipes_dataset.csv',index_col=0)

# transform ingredient field back into list when importing from CSV
recipes['ingredientLines'] = recipes['ingredientLines'].apply(
    lambda x: [item for item in x.split('\'') if item not in ('\,','[',']',', ')])

recipes.head()

,attributes.course,attributes.cuisine,attributes.holiday,attribution.html,attribution.logo,attribution.text,attribution.url,cookTime,cookTimeInSeconds,flavors.Bitter,...,nutritionEstimates,prepTime,prepTimeInSeconds,rating,source.sourceDisplayName,source.sourceRecipeUrl,source.sourceSiteUrl,totalTime,totalTimeInSeconds,yield
0,['Side Dishes'],['Italian'],NaN,<a href='http://www.yummly.com/recipe/Mushroom...,http://static.yummly.com/api-logo.png,Mushroom Risotto recipes: information powered ...,http://www.yummly.com/recipe/Mushroom-risotto-...,NaN,NaN,NaN,...,"[{'attribute': 'FAT_KCAL', 'unit': {'name': 'c...",NaN,NaN,5,Skinnytaste,http://www.skinnytaste.com/2009/10/risotto-is-...,http://www.skinnytaste.com,30 minutes,1800.0,servings: 6
1,['Main Dishes'],['Barbecue'],NaN,<a href='http://www.yummly.com/recipe/Filipino...,http://static.yummly.com/api-logo.png,Filipino BBQ Pork Skewers recipes: information...,http://www.yummly.com/recipe/Filipino-bbq-pork...,NaN,NaN,0.8333,...,"[{'attribute': 'FAT_KCAL', 'unit': {'name': 'c...",NaN,NaN,5,Skinnytaste,http://www.skinnytaste.com/2008/08/filipino-bb...,http://www.skinnytaste.com,40 min,2400.0,NaN
2,['Main Dishes'],['Italian'],NaN,<a href='http://www.yummly.com/recipe/Mushroom...,http://static.yummly.com/api-logo.png,Mushroom and Roasted Garlic Risotto recipes: i...,http://www.yummly.com/recipe/Mushroom-and-Roas...,NaN,NaN,1.0000,...,"[{'attribute': 'FAT_KCAL', 'unit': {'name': 'c...",NaN,NaN,3,MyRecipes,http://www.myrecipes.com/recipe/mushroom-roast...,http://www.myrecipes.com,1 Hr 25 Min,5100.0,Serves 6 (serving size: about 1 cup)
3,['Side Dishes'],"['French', 'American']",NaN,<a href='http://www.yummly.com/recipe/Gratin-D...,http://static.yummly.com/api-logo.png,Gratin Dauphinois (Scalloped Potatoes with Che...,http://www.yummly.com/recipe/Gratin-Dauphinois...,NaN,NaN,0.6667,...,"[{'attribute': 'FAT_KCAL', 'unit': {'name': 'c...",NaN,NaN,4,MyRecipes,http://www.myrecipes.com/recipe/gratin-dauphin...,http://www.myrecipes.com,55 min,3300.0,7 servings (serving size: 1 cup)
4,['Main Dishes'],['Barbecue'],NaN,<a href='http://www.yummly.com/recipe/Deliciou...,http://static.yummly.com/api-logo.png,Delicious Grilled Hamburgers recipes: informat...,http://www.yummly.com/recipe/Delicious-Grilled...,10 Min,600.0,0.1667,...,"[{'attribute': 'FAT_KCAL', 'unit': {'name': 'c...",5 Min,300.0,4,AllRecipes,http://allrecipes.com/Recipe/delicious-grilled...,http://www.allrecipes.com,15 Min,900.0,3 servings


### Extract Features We Care About

In [3]:
short_recipes = recipes[['attributes.course','attributes.cuisine','name','ingredientLines']]
short_recipes.to_csv('assets/short_recipes.csv')

## Ingredient Processing

There are a number of components to this. Some are done on an individual, token level. Others are done across ingredients:

**Token**:

- Strip numbers (quantities)
- Strip embedded numbers
- Strip common measurements and their abbreviations
- Strip punctuation
- Strip preparation methods
- Create custom stop word dictionary and remove those words from ingredient list

**Ingredient-level**:

- Inventory a "typical" pantry for expected, common items
    - Fuzzy match pantry items against the ingredients
    - Eliminate those ingredients to reduce recipe complexity
- Vectorize ingredients and apply agglomerative clustering
    - Measure similiarity/closeness of ingredients based on how they're described
    - Replace vectors of ingredients with vectors of clusters in representing recipes

### Preprocessing

In [4]:
# multi-threaded
def multi_process_ingredients(recipes,join=1,
                              nondescript=0,drop_words=None,
                              pantry=0,pantry_items=None):
    # create the search patterns

    # import punctuation characters
    # to remove all punctuation
    punct = string.punctuation
    punct_pattern = r"[{}]".format(punct)

    # to remove all numbers
    number_pattern = r"\d+\s"

    # embedded numbers
    embed_num_pattern = r".\d+."
    
    # removed prep methods
    prep_pattern = r"[a-z]+ed"
    
    # strip pluralization
    plural_pattern = r"s\s"
    
    # strip -ly
    ly_pattern = r"[a-z]+ly"
    
    # strip lead number
    lead_pattern = r"\d+[a-z]+"
    lead_repl = r"[a-z]+"
    
    # trail number
    trail_pattern = r"[a-z]+\d+"
    trail_repl = r"[a-z]+"
    
    recipes_ingredients = []
    ingredients = []

    for item in recipes:

        # strip punctuation
        text = re.sub(punct_pattern," ",item)
        # strip standalone numbers
        text = re.sub(number_pattern,"",text)
        # strip embedded numbers
        text = re.sub(embed_num_pattern,"",text)
        # strip preparation methods
        text = re.sub(prep_pattern,"",text)
        # strip pluralization
        text = re.sub(plural_pattern," ",text)
        # strip ly
        text = re.sub(ly_pattern,"",text)
        # lead
        text = re.sub(lead_pattern,lead_repl,text)
        # trail
        text = re.sub(trail_pattern,trail_repl,text)

        # tokenize
        tokenizer = RegexpTokenizer(r'\w+')
        processed_text = tokenizer.tokenize(text)

        # remove stop words
        processed_text = [text.lower() for text in processed_text if text.lower() 
                          not in stopwords.words('english')]
        
        # minimum word length
        processed_text = [text for text in processed_text if len(text) > 2]

        # remove non-descript recipe words
        if nondescript == 1 and drop_words != None:
            processed_text = [text.lower() for text in processed_text if text.lower()
                             not in drop_words]

        # append all each list that to describe an ingredient of the recipe
        ingredients.append(processed_text)

    # joined space-separated strings
    # attach all modifiers that describe each ingredient (non-separated)
    clean_ingredients = [" ".join(word) for word in ingredients]
    
    # remove pantry items
    if pantry == 1 and pantry_items != None:
        clean_ingredients = [text.lower() for text in clean_ingredients if text.lower() not in pantry_items]

    # append all ingredients for each recipe
    recipes_ingredients.append(clean_ingredients)   
    
        
    if join == 0:
        pass
    else:
        recipes_ingredients = [" ".join(ingredient) for ingredient in recipes_ingredients]
    
    return recipes_ingredients

In [5]:
num_cores = multiprocessing.cpu_count()
inputs = tqdm(short_recipes['ingredientLines'])

if __name__ == "__main__":
    recipes = Parallel(n_jobs=num_cores)(delayed(multi_process_ingredients)(i) for i in inputs)
print(len(recipes))

100%|██████████| 27638/27638 [01:38<00:00, 279.62it/s]


27638


In [6]:
# 1 list of ingredients for each recipe
recipes = [" ".join(recipe) for recipe in recipes]
recipes = pd.DataFrame(recipes)

In [7]:
recipes.head()

,0
0,cup baby bella mushroom cup arborio rice tsp o...
1,pork country style rib fat cut cubes cup soy ...
2,whole garlic heads tablespoon plu teaspoon ext...
3,garlic clove cooking spray potatoe cut inch sl...
4,pound lean ground beef tablespoon worcestershi...


### Word Counts for Custom Stopword Dictionary

#### Get Counts of Words that Describe Ingredients

In [8]:
# word counts
# get the words that occur most often in recipes
# these are candidates for removal in order to simplify the axis that we compare recipes

cvec = CountVectorizer(strip_accents=ascii)
cvecdata = cvec.fit_transform(recipes[0])

cvec_dense  = pd.DataFrame(cvecdata.todense(),
             columns=cvec.get_feature_names())

word_count = cvec_dense.sum(axis=0)    
cw = word_count.sort_values(ascending = False)
print(cw[0:10])

cw_dict = dict(cw)

cup           76725
teaspoon      46142
tablespoon    40078
pepper        26446
salt          26439
fresh         22512
ounce         18745
ground        18002
oil           17914
garlic        13556
dtype: int64


#### Manually Evaluate Ingredient Word List to Build Dictionary

In [9]:
# quick function to manually evaluate words that ought to be removed
# https://stackoverflow.com/questions/5844672/delete-an-item-from-a-dictionary

def removekey(d, key):
    r = dict(d)
    del r[key]
    return r

def eval_words(word_list):
    keeps = []
    nondescript = []
    
    nondescript_words = [] 
    keep_words = []
    
    for key,value in word_list.items():
            word_eval = input('Keep {}: {}, y or n?'.format(key,value))
        
            if word_eval == 'n':
                nondescript_words.append(key)
            else:
                keep_words.append(key)
            
            remaining_list = removekey(word_list,key)
            
            if len(nondescript_words) % 100 == 0:
                nondescript = nondescript + nondescript_words
                keeps = keeps + keep_words
                
                # empty holding lists
                keep_words = []
                nondescript_words = []
                
                prompt_continue = input('Continue: yes or no?')
                if prompt_continue == "yes":
                    pass
                else:
                    # export lists as pickles for recovery
                    # store outside the environment to limit reprocessing
                    words_lists = (keeps,nondescript,remaining_list)
                    names = ("keeps","nondescript","remaining")
                    for index,word in enumerate(words_lists):
                        with open("assets/"+names[index]+".pickle","wb") as file:
                            pickle.dump(word,file)
                    return keeps, nondescript,remaining_list
    
    # export word lists for recovery
    # so we don't have to do this multiple times
    words_lists = (keeps,nondescript,remaining_list)
    names = ("keeps","nondescript","remaining")
    for index,word in enumerate(words_lists):
        with open("assets/"+names[index]+".pickle","wb") as file:
            pickle.dump(word,file)
    
    return keep_words, nondescript_words, remaining_list

#### Load Pickled Keep/Drop Lists

In [10]:
# read in pickled results
keep_list = []
drop_list = []

names = ("_keeps","_nondescript")
for name in names:
    for index in range(6):
        with open("assets/"+str(index)+name+".pickle",'rb') as file_handle:
            if name == "_keeps":
                keep_list = keep_list + pickle.load(file_handle)
            elif name == "_nondescript":
                drop_list = drop_list + pickle.load(file_handle)

#### Fix Human Errors in Keep/Drop Lists

In [11]:
# fix human error from ingredient classifications

misclassed_words = ['dice','block','dipping','stems','liter','pestle','2lb','pad','addition','paleo',
                    'smaller','teaspoons','gf','meatles','anytime','xe4utet','almond','scallions',
                    'evoo','wing','non','meal','gala','escarole','nectarine','stuffing','ganache',
                    'speck','hefe','champignon','silver','blade','kabocha','goudak','lindt','quorn',
                    'choi','evoki','aioli','broil','drumette','tex','massamon','pao','steamer','dandelion',
                    'bonnet','rapini','cakes','yucatero','cheek','latin','jimmy','quahog','cone','durum',
                    'cornichons','banh','fryers','quantity','5tbsp','llime','chopping','spam','ink','plant',
                    'triangular','valencia','tubetti','tubettini','cavatelli','perhap','livers','bee',
                    'tartine','teacup','barlett','maker','xlour','jell','fat','free','package'
                   ]

for word in misclassed_words:
    if word in keep_list and word not in drop_list:
        drop_list.append(word)
        keep_list.remove(word)
        print('{} added to drop_list'.format(word))
    elif word in drop_list and word not in keep_list:
        keep_list.append(word)
        drop_list.remove(word)
        print('{} added to keep list'.format(word))
    elif word in drop_list and word in keep_list:
        print('! {} found on both lists !')
    elif word not in drop_list and word not in keep_list:
        print('! {} not found on either list ! You misspelled target word'.format(word))
    else:
        print('! Bigger problems !')

with open("assets/keep_list.pickle","wb") as file:
    pickle.dump(keep_list,file)
with open("assets/drop_list.pickle","wb") as file:
    pickle.dump(drop_list,file)

dice added to drop_list
block added to drop_list
dipping added to drop_list
stems added to drop_list
liter added to drop_list
pestle added to drop_list
2lb added to drop_list
pad added to keep list
addition added to drop_list
paleo added to drop_list
smaller added to drop_list
teaspoons added to drop_list
gf added to drop_list
meatles added to drop_list
anytime added to drop_list
xe4utet added to drop_list
almond added to keep list
scallions added to keep list
evoo added to keep list
wing added to keep list
non added to keep list
meal added to keep list
gala added to keep list
escarole added to keep list
nectarine added to keep list
stuffing added to keep list
ganache added to keep list
speck added to keep list
hefe added to keep list
champignon added to keep list
silver added to keep list
blade added to keep list
kabocha added to keep list
goudak added to keep list
lindt added to keep list
quorn added to keep list
choi added to keep list
! evoki not found on either list ! You misspell

We will feed this *drop_list* back into the preprocessing function to remove the non-descript words from the ingredient descriptions

### Identifying Ingredients in our Pantry

In [12]:
# re-process ingredient list, this time removing the non-descript words identified above
# getting data ready to identify similiarity with our pantry items

num_cores = multiprocessing.cpu_count()
inputs = tqdm(short_recipes['ingredientLines'])

if __name__ == "__main__":
    recipes_drops = Parallel(n_jobs=num_cores)(delayed(multi_process_ingredients)(i,join=0,
                                                                                  nondescript=1,
                                                                                  drop_words=drop_list
                                                                                 )
                                                                            for i in inputs)
print(len(recipes_drops))

100%|██████████| 27638/27638 [01:49<00:00, 252.35it/s]


27638


#### Pantry Items

I inventoried my kitchen to build a baseline set of ingredients we can expect to find in a kitchen.

These ingredients are matched to the master list of ingredients, using fuzzy matching, and matches are dropped out of the ingredient list to reduce dimensionality.

In [13]:
pantry = ['oregano','garlic powder','ground cumin','onion powder','ground mustard','hot hungarian paprika',
          'mexican oregano','smoked paprika','dill weed','ground turmeric','ground ginger','ground cloves',
         'cumin seed','cayenne pepper','chili powder','ground thyme','celery seed','curry powder',
          'ground white pepper','paprika','ground nutmeg','old bay','maple syrup','thyme leaves',
          'ground black pepper','black pepper','black peppercorns','crushed red pepper flakes','whole oregano',
         'minced onion','fennel seed','cinnamon','dried basil','anise seed','bay leaves','bay leaf',
          'ancho chili powder','ground cloves','coriander','vanilla extract','italian seasoning',
          'apple cider vinegar','honey','corn starch','balsamic vinegar','bread crumbs','white wine vinegar',
         'soy sauce','ketchup','tomato ketchup','red wine vinegar','vegatable oil','canola oil','sherry',
          'baking powder','baking soda','molasses','peanut butter','olive oil','extra virgin olive oil','salt',
          'sea salt','kosher salt','white vinegar','egg','eggs','egg whites','egg yolk','brown sugar','sugar',
          'flour','evoo','butter','salt pepper','garlic','garlic clove']

#### Pantry Processing

In [14]:
# this eliminates ingredients that have been wholly reduced to blanks
test = [[[ingredient for 
                   ingredient in recipe if ingredient != ''] 
                  for recipe in item] 
                 for item in recipes_drops]

# we're going to take all the ingredients from every recipe and string them together
# then take the set of that to find every unique ingredient

ingredient_master = []
for items in test:
    for recipe in items:
        ingredient_master = list(set(ingredient_master + recipe))
print(len(ingredient_master))

32983


In [15]:
# from the ingredient_master list, we eliminate ingredients that bear substantial similiarity to our pantry items
# because these ingredients are IN our pantry, they are not essential for determining overall recipe similiarity
# in fact, it gives us more degrees of freedom to find a match by increasing the range of possible flavor profiles
# of a related recipe -- because we go into the pantry and pull out different spices other than those in our target
# recipe
# this will fuzzy matching (ratio)

def match_pantry(ingredient_list,pantry_items=pantry,n_jobs=0):
    if n_jobs == 1:
        if ingredient_list in pantry_items:
            return ingredient_list
        else:
            for item in pantry_items:
                if fuzz.ratio(item,ingredient_list) > 70:
                    return ingredient_list
                    break
                else:
                    pass
    else:
        pantry_matches = []

        for ingredient in ingredient_list:
            if ingredient in pantry_items:
                pantry_matches.append(ingredient)
            else:
                for item in pantry_items:
                    if fuzz.ratio(item,ingredient) > 70:
                        pantry_matches.append(ingredient)
                        break
        return pantry_matches  

In [16]:
num_cores = multiprocessing.cpu_count()
inputs = tqdm(ingredient_master)

if __name__ == "__main__":
    pantry_matches = Parallel(n_jobs=num_cores)(delayed(match_pantry)(i,pantry,n_jobs=1) for i in inputs)
    
pantry_matches = [match for match in pantry_matches if match != None]

with open("assets/pantry_matches.pickle","wb") as file:
    pickle.dump(pantry_matches,file)

100%|██████████| 32983/32983 [00:06<00:00, 5322.38it/s]


### Reducing Ingredient Dimensionality

After doing all the above preprocessing steps on the ingredients, we still need to reduce the dimensionality of the ingredients that describe the recipes in the cookbook.

My solution is to take the list of each unique ingredient, CountVectorize them, and feed them into an agglomerative clustering algorithm. 

With our cluster assignments, we will replace ingredients to be represented by the cluster that they belong to. Our recipes will now be able to be represented as a vector of clusters instead a vector of ingredients.

The dimensionality of ingredient will have then been reduced, and we can move into the recipe recommender and measuring the similiarity between recipes.

In [17]:
# re-process ingredient list, this time removing the non-descript words and pantry items as identified above
# getting data ready for clustering

num_cores = multiprocessing.cpu_count()
inputs = tqdm(short_recipes['ingredientLines'])

if __name__ == "__main__":
    cookbook = Parallel(n_jobs=num_cores)(delayed(multi_process_ingredients)(i,join=0,
                                                                                  nondescript=1,
                                                                                  drop_words=drop_list,
                                                                                  pantry=1,
                                                                                  pantry_items=pantry_matches
                                                                                 )
                                                                            for i in inputs)

100%|██████████| 27638/27638 [01:52<00:00, 245.70it/s]


In [18]:
# this eliminates ingredients that have been wholly reduced to blanks
test = [[[ingredient for 
                   ingredient in recipe if ingredient != ''] 
                  for recipe in item] 
                 for item in cookbook]

# we're going to take all the ingredients from every recipe and string them together
# then take the set of that to find every unique ingredient
ingredient_master = []
for items in test:
    for recipe in items:
        ingredient_master = list(set(ingredient_master + recipe))
print(len(ingredient_master))

31266


#### Vectorize Ingredients

In [19]:
ingredient_master_cvector = CountVectorizer(strip_accents=ascii)
ingredient_master_cvectordata = ingredient_master_cvector.fit_transform(ingredient_master)

ingredient_master_cvector_dense  = pd.DataFrame(ingredient_master_cvectordata.todense(),
             columns=ingredient_master_cvector.get_feature_names())

ingredient_master_cvector_dense.shape

(31266, 7265)

#### Agglomerative Clustering of Vectorized Ingredients

In [ ]:
def plot_dendogram(df):
    
    # Data preparation:
    X = df.values
    Z = linkage(X, 'ward')
    
    # Plotting:
    plt.title('Dendrogram')
    plt.xlabel('Index Numbers')
    plt.ylabel('Distance')
    dendrogram(
        Z,
        leaf_rotation=90.,  
        leaf_font_size=8.,
    )
    plt.show()
    
    
plot_dendogram(ingredient_master_cvector_dense)

In [ ]:
sns.set_style("darkgrid")
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap
import matplotlib.cm as cm



def plot_dist_thresh(max_dist=200):
    # max_dist = 200 # Pairwise distance.
    plot_dendogram(lang)
    clusters = fcluster(Z, max_dist, criterion='distance')
    
    print("Clusters represented at distance: ", set(clusters))
    
    # Complete color maps from Matplotlib.
    
    # Plotting.
    # Add a legend with some customizations.
    
    fig, ax = plt.subplots(1, 2, figsize=(10, 4))

    ax[0].scatter(lang.index, X[:,5], c=clusters, cmap=cm.jet, s=40)
    ax[0].set_title("Max Dist: %d" % max_dist)
    plt.legend(clusters, loc='upper right', shadow=True, scatterpoints=1)
    ax[0].legend(['c{}'.format(i) for i in range(len(clusters))], loc=2, bbox_to_anchor=(1.05, 1),
                 borderaxespad=0., fontsize=11)
        
    t = (0, max_dist)
    ax[1].plot((0, 200), (max_dist, max_dist), 'r--')
    ax[1].set_title('Dendrogram')
    ax[1].set_xlabel('Index Numbers')
    ax[1].set_ylabel('Distance')
    R = dendrogram(
        Z,
        leaf_rotation=90.,  
        leaf_font_size=8.,
        ax=ax[1]
        #link_color_func=lambda color: cmaps['Miscellaneous'],
    )
    return None
    
def plot_wrapper(max_dist):
    plot_dist_thresh(max_dist)

### Represent Recipes as Vectors of Clusters, Instead of Vectors of Ingredients

## Recommender

### Prep Data for Recommender

In [ ]:
cookbook_df = pd.DataFrame(cookbook)
cookbook_df.rename(columns={0:'cookbook'},inplace=True)

In [ ]:
# drop out ingredients that have been reduced to blanks after stop words

cookbook_pipe = [[[ingredient for 
                   ingredient in recipe if ingredient != ''] 
                  for recipe in item] 
                 for item in cookbook]

# collapse list of list of ingredients into 
# pipe-separated list of ingredients to feed to tokenizer

cookbook_pipe = [["|".join(ingredient) for 
                        ingredient in recipe] 
                       for recipe in cookbook]

### Recommender

In [ ]:
def mean_center_rows(df):
    return (df.T - df.mean(axis=1)).T

In [ ]:
recipes_mc = mean_center_rows(ingredient_cvector_dense)

# check for nulls
print('nulls: {}'.format(recipes_mc.isnull().sum().sum()))

In [ ]:
sim_matrix = cosine_similarity(recipes_mc)
recipe_sim = pd.DataFrame(sim_matrix, columns=recipes_mc.index, index=recipes_mc.index)

In [ ]:
sns.heatmap(recipe_sim, annot=True, cmap='coolwarm')